# Aula 06 &mdash; Data Scraping com API do Twitter

Renato Vimieiro
rv2 {em} cin.ufpe.br

março 2017

In [16]:
import pandas as pd
import numpy as np

import tweepy

import configparser

## Introdução

Nas aulas anteriores vimos como obter dados diretamente de uma página web. Para isso, utilizamos a biblioteca BeautifulSoup. A biblioteca facilitava a busca de tags e, consequentemente, de informações úteis nas páginas. Embora tenhamos que lançar mão desse recurso em muitas ocasiões, alguns sites fornecem APIs para obtermos dados de forma semi-estruturada. Esses dados possuem formato livre, mas possuem indicadores do conteúdo. As representações mais usuais são [XML](https://en.wikipedia.org/wiki/XML) e [JSON](https://en.wikipedia.org/wiki/JSON).

O formato JSON tem se tornado um dos mais usados para troca de dados na internet. Ele tem substituido inclusive XML. Muitos bancos de dados NoSQL, como MongoDB, fornecem suporte direto a dados nesse formato. Em outras palavras, JSON é, talvez, o formato franco de representação de dados para Data Science. Sendo assim, discutiremos rapidamente sua representação.

## JSON

O formato JSON (lê-se como *jay-son*) é essencialmente um dicionário. Ele descreve uma sequência de pares (chave,valor). Objetos no formato JSON são delimitados por { e }. Por sua vez, os pares são separados por vírgulas. As chaves e valores são separados por : (dois-pontos).

As chaves devem ser sempre strings em unicode. Os valores podem ser:

- string
- número
- objeto
- vetor
- true
- false
- null

Os vetores referenciados acima são como vetores/listas em uma linguagem de programação. Eles podem conter uma sequência de um ou mais valores, incluindo outros vetores e objetos. A gramática do formato pode ser conferida em http://www.json.org/json-pt.html.

### Exemplo de um dado no formato JSON

O exemplo a seguir foi retirado da Wikipedia. Ele ilustra como poderiam ser armazenados diversos dados sobre uma pessoa. A escolha das chaves é completamente arbitrária e definida pelo desenvolvedor.

```json
{
  "firstName": "John",
  "lastName": "Smith",
  "isAlive": true,
  "age": 25,
  "address": {
    "streetAddress": "21 2nd Street",
    "city": "New York",
    "state": "NY",
    "postalCode": "10021-3100"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "212 555-1234"
    },
    {
      "type": "office",
      "number": "646 555-4567"
    },
    {
      "type": "mobile",
      "number": "123 456-7890"
    }
  ],
  "children": [],
  "spouse": null
}
```

## API do Twitter

O Twitter oferece aos desenvolvedores uma API para permitir a interação entre o site e aplicativos de terceiros. A API é uma das ricas entre as redes sociais e, por isso, muitas pesquisas comerciais e científicas usam a plataforma.

A API do Twitter se subdivide em outras duas APIs: REST e Streaming. A API de REST permite aos desenvolvedores acessar os perfis dos usuários (públicos) assim como seus seguidores, listas, timeline, além de efetuar buscas por termos ou usuários na plataforma. A API de Streaming, por outro lado, é dedicada ao monitoramento em tempo real dos tweets sendo postados na plataforma.

Além dessas duas interfaces públicas, o Twitter ainda oferece uma terceira interface comercial, Gnip, para acesso a grandes volumes de dados. O Gnip, basicamente, fornece acesso à base de dados (pública) inteira do Twitter. Enquanto a API de Streaming faz amostragem dos tweets em tempo real (não é muito claro como isso é feito), o Gnip fornece acesso a efetivamente todos as mensagens sendo postadas. O mesmo acontece em relação ao REST, que limita, por exemplo, a busca por termos, não retornando todos os resultados. O Gnip, ao contrário, permite efetuar uma busca em toda a base.

## Acesso à API do Twitter

O acesso à API requer a criação de um perfil na plataforma e registro de um aplicativo. A autenticação é feita através do protocolo [OAuth](https://en.wikipedia.org/wiki/OAuth). A ideia básica do protocolo é que um usuário possa delegar acesso a recursos em um determinado site a terceiros sem revelar suas credenciais. Resumindo, o protocolo permite que chaves de acesso sejam geradas pelo usuário; essas chaves são passadas a um terceiro, a quem se deseja permitir acesso aos recursos do usuário; finalmente, a parte a quem o acesso foi delegado pode conectar-se ao sistema usando as chaves que lhe foram enviadas.

#### Registrando um novo aplicativo

Para registrar um aplicativo e gerar novas chaves de acesso, o usuário deve ir até o site https://apps.twitter.com.

![https://apps.twitter.com](images/tela_inicial_apps_twitter.png)

Após logar, a tela inicial mostrará as aplicações autorizadas no momento.

![https://apps.twitter.com](images/apps_twitter_logado.png)

Para criar/autorizar um novo aplicativo, clique no botão **Create New App**. Em seguida, você será redirecionado para uma página onde deverá preencher os dados do aplicativo

![https://apps.twitter.com](images/create_new_app.png)

e aceitar os termos e condições de uso

![https://apps.twitter.com](images/create_new_app_accept_terms_confirm.png)

Pronto! Agora você já resgistrou o aplicativo e poderá gerar as chaves de acesso como a seguir.

#### Obtendo as chaves de acesso

Após registrar o novo aplicativo, você será redirecionado à página de gerenciamento do aplicativo

![https://apps.twitter.com](images/new_app_twitter.png)

Configure as permissões da maneira como for mais conveniente. Em geral, para apenas minerar os dados, basta permitir que o aplicativo possa somente ler os dados. Isso evita, ou diminui as chances, de postagens indevidas caso alguém obtenha suas chaves.

Para gerar as chaves de acesso, vá até a aba **Keys and Access Tokens**.

![https://apps.twitter.com](images/consumer_key_twitter.png)

Inicialmente o aplicativo não está autorizado a logar em nome do usuário. Você verá que foram assinalados um identificador e uma chave de acesso de usuário (*Consumer key and secret*), como mostrado na figura acima. Essas chaves servem para autenticar o usuário, contudo, elas não autenticam o aplicativo, como mostrado na figura abaixo.

![https://apps.twitter.com](images/authorize_app_twitter.png)

Para autorizar o aplicativo, clique no botão **Create my access tokens**. O Twitter então gerará novos identificadores e chaves de acesso para o aplicativo (*Access token and secret*). Esses deverão ser usados em conjunto com as chaves de usuário para se ter acesso à API.

![https://apps.twitter.com](images/access_tokens_twitter.png)

É importante lembrar que esses dois pares de identificadores e chaves nunca devem estar legíveis em seu programa. Eles são como senhas e, portanto, devem ser protegidos.

Agora que já obtivemos as chaves de acesso, podemos iniciar a coleta de dados.

## Acessando a API com Tweepy

Existem diversas bibliotecas de Python para acessar a API do Twitter, como listado na página dos desenvolvedores (https://dev.twitter.com/resources/twitter-libraries). As duas bibliotecas mais usadas (pelo que vi na internet) são [Python-Twitter](https://github.com/bear/python-twitter) e [Tweepy](https://github.com/tweepy/tweepy). No entanto, Tweepy está melhor documentada e possui melhor suporte à API de Streaming. Assim, usaremos Tweepy em nossos exemplos.

#### Conectando-se a API com Tweepy

A biblioteca Tweepy, na verdade, é um wrapper para os serviços listados na API do Twitter. Ela se encarrega dos detalhes de codificação e requisições ao servidor, abstraindo as consultas através de funções e métodos. A biblioteca abstrai inclusive os detalhes de autenticação.

O primeiro passo para coletar os dados é autenticar o usuário e o aplicativo. Isso é feito da seguinte forma:


In [12]:
# As chaves estao armazenadas em um arquivo de configuracoes
# Python possui uma biblioteca para processar esses arquivos (ver configparser)
config = configparser.ConfigParser()
config.read('../configs.txt')
chaves = config['DEFAULT']

access_token = "528007904-I2R5QcOVAkEjUBb4eRoUOZgpSM3AbdyoD7SZcgko"
access_token_secret = "u50HJYk2Q8xTlSTuFq8mKF9jLIrL9izy2kZO8TwvyOjRp"
consumer_key = "zrcVNSqm0GXf2OtIYtb66IoOX"
consumer_secret = "HtxEdoDi5ftRkXduKuzZvIqF3fi9v1yp0yQNVWXRxWoX2hNTLh"

# Autentica o usuario na API
#auth = tweepy.OAuthHandler(chaves["consumer_key"], chaves["consumer_secret"])
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)

# Autentica o aplicativo
#auth.set_access_token(chaves["access_token"], chaves["access_token_secret"])
auth.set_access_token(access_token, access_token_secret)

O objeto `auth` contém a sessão autenticada e deverá ser usado nas conexões com a API.

#### Acesso à REST API

O acesso à REST API é feito através de objetos da classe `tweepy.API`. Os objetos dessa classe possuem métodos que encapsulam os acessos às funcionalidades da API. Assim, devemos instaciar a classe usando a conexão criada anteriormente. É importante respeitar os limites impostos pela API, caso contrário, podemos ser bloqueados. As requisições que ultrapassarem os limites do Twitter resultaram em exceções do tipo `tweepy.RateLimitError`. Tweepy pode controlar automaticamente esses acessos. Podemos estabelecer que as chamadas a determinados métodos devem esperar até que o limite de requisições seja reestabelecido. O limite de requisições varia dependendo do tipo de requisição. O Twitter também faz constantes modificações nesses limites. Então, você sempre deve consultar a documentação da API para se certificar dos limites atuais. Eles podem ser conferidos em https://dev.twitter.com/rest/public/rate-limits.

In [13]:
api = tweepy.API(auth, wait_on_rate_limit=False)

O objeto `api` agora é nossa interface de acesso ao Twitter.

Podemos usá-la para saber quais são os trending topics nos diversos lugares onde essa informação é monitorada.

In [17]:
api.trends_available()

[{'country': '',
  'countryCode': None,
  'name': 'Worldwide',
  'parentid': 0,
  'placeType': {'code': 19, 'name': 'Supername'},
  'url': 'http://where.yahooapis.com/v1/place/1',
  'woeid': 1},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Winnipeg',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/2972',
  'woeid': 2972},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Ottawa',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3369',
  'woeid': 3369},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Quebec',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3444',
  'woeid': 3444},
 {'country': 'Canada',
  'countryCode': 'CA',
  'name': 'Montreal',
  'parentid': 23424775,
  'placeType': {'code': 7, 'name': 'Town'},
  'url': 'http://where.yahooapis.com/v1/place/3534',

In [18]:
trendtopics = api.trends_place(455824)
trendtopics

[{'as_of': '2017-03-29T14:19:17Z',
  'created_at': '2017-03-29T14:09:20Z',
  'locations': [{'name': 'Recife', 'woeid': 455824}],
  'trends': [{'name': '#WelcomeBack2BrazilJB',
    'promoted_content': None,
    'query': '%23WelcomeBack2BrazilJB',
    'tweet_volume': 325107,
    'url': 'http://twitter.com/search?q=%23WelcomeBack2BrazilJB'},
   {'name': '#MasterChefBR',
    'promoted_content': None,
    'query': '%23MasterChefBR',
    'tweet_volume': 84009,
    'url': 'http://twitter.com/search?q=%23MasterChefBR'},
   {'name': 'Diego Souza',
    'promoted_content': None,
    'query': '%22Diego+Souza%22',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=%22Diego+Souza%22'},
   {'name': 'Recife',
    'promoted_content': None,
    'query': 'Recife',
    'tweet_volume': None,
    'url': 'http://twitter.com/search?q=Recife'},
   {'name': 'Daniel',
    'promoted_content': None,
    'query': 'Daniel',
    'tweet_volume': 118165,
    'url': 'http://twitter.com/search?q=Daniel'},


Como pode ser visto, o resultado é uma lista de objetos no formato JSON (um dicionário). Podemos usar Pandas para criar um data frame com os dados obtidos. Repare que a requisição retorna um único objeto com quatro chaves: data da apuração, data da requisição, local, e os tópicos. Nossa intenção é criar um data frame com os dados dos tópicos. As outras informações são complementares e podem ser replicadas no data frame se necessário após a criação.

In [26]:
dfTrendTopics = pd.DataFrame(trendtopics[0]['trends'])

In [29]:
dfTrendTopics.head()

,name,promoted_content,query,tweet_volume,url
0,#WelcomeBack2BrazilJB,None,%23WelcomeBack2BrazilJB,325107.0,http://twitter.com/search?q=%23WelcomeBack2Bra...
1,#MasterChefBR,None,%23MasterChefBR,84009.0,http://twitter.com/search?q=%23MasterChefBR
2,Diego Souza,None,%22Diego+Souza%22,NaN,http://twitter.com/search?q=%22Diego+Souza%22
3,Recife,None,Recife,NaN,http://twitter.com/search?q=Recife
4,Daniel,None,Daniel,118165.0,http://twitter.com/search?q=Daniel


In [40]:
dfTrendTopics.sort_values(by='tweet_volume',ascending=False).\
                head(10)[['name','tweet_volume']]

,name,tweet_volume
11,Brexit,556589.0
0,#WelcomeBack2BrazilJB,325107.0
15,Stories,231358.0
36,#몬스타엑스,160275.0
4,Daniel,118165.0
1,#MasterChefBR,84009.0
13,Galaxy S8,33053.0
48,#Highlight2ndWin,22697.0
16,Only You,15988.0
14,Bob Dylan,13432.0


Podemos também obter a lista de tweets de um usuário

In [43]:
trumpsTweets = api.user_timeline(screen_name='realdonaldtrump')
trumpsTweets[0]

Status(_json={'favorited': False, 'geo': None, 'is_quote_status': False, 'coordinates': None, 'id_str': '847061031293779969', 'retweet_count': 7674, 'in_reply_to_user_id_str': None, 'in_reply_to_status_id_str': None, 'entities': {'urls': [], 'hashtags': [], 'symbols': [], 'user_mentions': []}, 'retweeted': False, 'contributors': None, 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!', 'created_at': 'Wed Mar 29 12:21:02 +0000 2017', 'in_reply_to_status_id': None, 'place': None, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_screen_name': None, 'in_reply_to_user_id': None, 'favorite_count': 35385, 'user': {'has_extended_profile': False, 'notifications': False, 'location': 'Washington, DC', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/1980294624/DJT_Headshot_V2_normal.jpg', 'profile_sidebar_fill_color': 'C5CEC0', 'id_str'

O resultado da chamada é uma lista de objetos da classe `Status`. Inspecionando esse objeto, vemos que ele possui um atributo `_json` que armazena o tweet como um dicionário (JSON).

In [46]:
print(type(trumpsTweets[0]._json))
trumpsTweets[0]._json

<class 'dict'>


{'contributors': None,
 'coordinates': None,
 'created_at': 'Wed Mar 29 12:21:02 +0000 2017',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'favorite_count': 35385,
 'favorited': False,
 'geo': None,
 'id': 847061031293779969,
 'id_str': '847061031293779969',
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'is_quote_status': False,
 'lang': 'en',
 'place': None,
 'retweet_count': 7674,
 'retweeted': False,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!',
 'truncated': False,
 'user': {'contributors_enabled': False,
  'created_at': 'Wed Mar 18 13:46:38 +0000 2009',
  'default_profile': False,
  'default_profile_image': False,
  'description': '45th Pre

Essa representação reflete o modelo de objetos estabelecido pela API do Twitter. Um tweet possui vários campos, como descrito em https://dev.twitter.com/overview/api/tweets. Nem todos os campos são interessantes para a análise de dados. Por exemplo, o tweet contém informações detalhadas sobre o usuário, como o link para a imagem de seu perfil, o link para a imagem que ele usa de fundo, entre outras coisas. Geralmente, esses dados não são úteis para análise. Assim, devemos ser cuidadosos para criar um data frame somente com os dados que julgarmos importantes.

Por exemplo, podemos selecionar os seguintes campos:

- id
- created_at
- entities (hashtags, urls, user_mentions)
- favorite_count
- in_reply_to_screen_name
- in_reply_to_status_id
- in_reply_to_user_id
- lang
- place (country, name)
- retweet_count
- source
- text
- truncated
- user (screen_name, id)

In [67]:
def filtra_campos(tweet):    
    dic = {a[0] : a[1] for a in tweet.items() if a[0] in ['id',
                                    'created_at',
                                    'favorite_count',
                                    'in_reply_to_screen_name',
                                    'in_reply_to_status_id',
                                    'in_reply_to_user_id',
                                    'lang',
                                    'retweet_count',
                                    'source',
                                    'text',
                                    'truncated']}
    dic.update(tweet['entities'])
    if tweet['place']:
        dic.update({'place_'+a[0] : a[1] for a in tweet['place'].items() \
                    if a[0] in ['country', 'name']})
    dic.update({'user_'+a[0] : a[1] for a in tweet['user'].items() \
                if a[0] in ['screen_name', 'id']})
    return dic
    

In [68]:
filtra_campos(trumpsTweets[0]._json)

{'created_at': 'Wed Mar 29 12:21:02 +0000 2017',
 'favorite_count': 35385,
 'hashtags': [],
 'id': 847061031293779969,
 'in_reply_to_screen_name': None,
 'in_reply_to_status_id': None,
 'in_reply_to_user_id': None,
 'lang': 'en',
 'retweet_count': 7674,
 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>',
 'symbols': [],
 'text': 'If the people of our great country could only see how viciously and inaccurately my administration is covered by certain media!',
 'truncated': False,
 'urls': [],
 'user_id': 25073877,
 'user_mentions': [],
 'user_screen_name': 'realDonaldTrump'}

In [70]:
dfTrumpsTweets = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),trumpsTweets)))
dfTrumpsTweets.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Wed Mar 29 12:21:02 +0000 2017,35385,[],847061031293779969,None,None,None,en,NaN,7674,"<a href=""http://twitter.com/download/iphone"" r...",[],If the people of our great country could only ...,False,[],25073877,[],realDonaldTrump
1,Wed Mar 29 12:01:52 +0000 2017,30505,[],847056211006631936,None,None,None,en,NaN,6779,"<a href=""http://twitter.com/download/iphone"" r...",[],Remember when the failing @nytimes apologized ...,False,[],25073877,"[{'id_str': '807095', 'id': 807095, 'indices':...",realDonaldTrump
2,Tue Mar 28 22:41:09 +0000 2017,64865,[],846854703183020032,None,None,None,en,NaN,19235,"<a href=""http://twitter.com/download/iphone"" r...",[],Why doesn't Fake News talk about Podesta ties ...,False,[],25073877,"[{'id_str': '1367531', 'id': 1367531, 'indices...",realDonaldTrump
3,Tue Mar 28 21:48:40 +0000 2017,34999,"[{'indices': [31, 44], 'text': 'MadeInTheUSA'}]",846841493952319489,None,None,None,en,"[{'indices': [83, 106], 'media_url_https': 'ht...",7521,"<a href=""http://twitter.com/download/iphone"" r...",[],A NEW ERA IN AMERICAN ENERGY! \n#MadeInTheUSA🇺...,False,"[{'indices': [59, 82], 'url': 'https://t.co/EG...",25073877,[],realDonaldTrump
4,Tue Mar 28 20:57:17 +0000 2017,30731,[],846828561491202048,None,None,None,en,NaN,5840,"<a href=""http://twitter.com/download/iphone"" r...",[],It was an honor to welcome @GLFOP to the @Whit...,True,"[{'indices': [121, 144], 'url': 'https://t.co/...",25073877,"[{'id_str': '222196210', 'id': 222196210, 'ind...",realDonaldTrump


Podemos ainda pesquisar tweets usando termos ou *hashtags* através da interface de pesquisa.

```python
API.search(q[, lang][, locale][, rpp][, page][, since_id][, geocode][, show_user])
```

In [72]:
tweetsDataScience = api.search('#datascience')
tweetsDataScience

[Status(_json={'favorited': False, 'geo': None, 'is_quote_status': False, 'coordinates': None, 'id_str': '847171703511961602', 'retweet_count': 0, 'in_reply_to_user_id_str': None, 'in_reply_to_status_id_str': None, 'entities': {'urls': [{'indices': [76, 99], 'url': 'https://t.co/yfhNsDescQ', 'expanded_url': 'http://ift.tt/1nAEfXW', 'display_url': 'ift.tt/1nAEfXW'}], 'hashtags': [{'indices': [100, 112], 'text': 'datascience'}, {'indices': [113, 122], 'text': 'startups'}], 'symbols': [], 'user_mentions': []}, 'retweeted': False, 'contributors': None, 'text': 'Census Analyzer: preview of technology for web based tool for data analysis https://t.co/yfhNsDescQ #datascience #startups', 'created_at': 'Wed Mar 29 19:40:48 +0000 2017', 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'in_reply_to_status_id': None, 'place': None, 'source': '<a href="http://datasciencepakistan.com" rel="nofollow">Data Pakistan</a>', 'in_reply_to_screen_name': None, 'in_reply_to_user_id': None, 'l

O resultado novamente é uma lista de objetos do tipo `Status`, a qual podemos processar da mesma forma como fizemos com os tweets do Trump.

In [74]:
dfTweetsDataScience = pd.DataFrame(list(map(lambda x: filtra_campos(x._json),tweetsDataScience)))
dfTweetsDataScience.head()

,created_at,favorite_count,hashtags,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,media,retweet_count,source,symbols,text,truncated,urls,user_id,user_mentions,user_screen_name
0,Wed Mar 29 19:40:48 +0000 2017,0,"[{'indices': [100, 112], 'text': 'datascience'...",847171703511961602,None,None,None,en,NaN,0,"<a href=""http://datasciencepakistan.com"" rel=""...",[],Census Analyzer: preview of technology for web...,False,"[{'indices': [76, 99], 'url': 'https://t.co/yf...",1480718904,[],ImDataScientist
1,Wed Mar 29 19:40:29 +0000 2017,0,"[{'indices': [17, 29], 'text': 'IoTSecurity'},...",847171622134009856,None,None,None,en,NaN,50,"<a href=""http://www.flipboard.com"" rel=""nofoll...",[],RT @reach2ratan: #IoTSecurity Threat Map #Cy...,False,[],49714613,"[{'id_str': '262641807', 'id': 262641807, 'ind...",pfarrian
2,Wed Mar 29 19:40:24 +0000 2017,0,"[{'indices': [90, 103], 'text': 'DeepLearning'...",847171604513570816,None,None,None,en,NaN,14,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @MikeTamir: CrowdAI sells artificial-intell...,False,"[{'indices': [66, 89], 'url': 'https://t.co/Tr...",4899161614,"[{'id_str': '331659804', 'id': 331659804, 'ind...",SalesTechIO
3,Wed Mar 29 19:40:12 +0000 2017,0,"[{'indices': [46, 58], 'text': 'DataScience'},...",847171551573299205,None,None,None,en,NaN,0,"<a href=""http://twitter.com/download/android"" ...",[],Simplifying The Fourth Industrial Revolution: ...,False,"[{'indices': [87, 110], 'url': 'https://t.co/G...",798179779979546624,"[{'id_str': '91478624', 'id': 91478624, 'indic...",enricfilba
4,Wed Mar 29 19:40:05 +0000 2017,0,"[{'indices': [31, 42], 'text': 'Algorithms'}, ...",847171522364104704,None,None,None,en,NaN,112,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",[],RT @KirkDBorne: Top Prediction #Algorithms in ...,False,"[{'indices': [59, 82], 'url': 'https://t.co/DH...",393706761,"[{'id_str': '534563976', 'id': 534563976, 'ind...",ARC_UM


Agora podemos iniciar nosso processamento. Podemos, por exemplo, identificar as mensagens mais interessantes, considerando o número de retweets.

In [75]:
dfTweetsDataScience.retweet_count.describe()

count     15.000000
mean      23.066667
std       37.484219
min        0.000000
25%        1.000000
50%        4.000000
75%       18.000000
max      112.000000
Name: retweet_count, dtype: float64

In [81]:
dfTweetsDataScience[dfTweetsDataScience.retweet_count >= 18].sort_values(by='retweet_count')['text']

12    RT @ipfconline1: 40 Techniques Used by Data Sc...
14    RT @ipfconline1: 40 Techniques Used by Data Sc...
1     RT @reach2ratan: #IoTSecurity Threat Map   #Cy...
7     RT @MktgSciences: Tips and Tricks for Boosting...
4     RT @KirkDBorne: Top Prediction #Algorithms in ...
Name: text, dtype: object